In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from optimus import Optimus
from optimus.profiler.functions import print_json as j

In [3]:
# Create optimus
op = Optimus(master="local", app_name= "optimus")


             ____        __  _                     
            / __ \____  / /_(_)___ ___  __  _______
           / / / / __ \/ __/ / __ `__ \/ / / / ___/
          / /_/ / /_/ / /_/ / / / / / / /_/ (__  ) 
          \____/ .___/\__/_/_/ /_/ /_/\__,_/____/  
              /_/                                  
              
Just checking that all necessary environments vars are present...
-----
PYSPARK_PYTHON=python
SPARK_HOME=C:\opt\spark\spark-2.3.1-bin-hadoop2.7
JAVA_HOME=C:\java8
-----
Starting or getting SparkSession and SparkContext...
Setting checkpoint folder ( local ). If you are in a cluster initialize optimus with master='your_ip' as param
Deleting previous folder if exists...
Creating the checkpoint directory...
Optimus successfully imported. Have fun :).


In [9]:
df = op.read.csv("Meteorite_Landings.csv", header=True)

In [5]:
df.show(10)

+-------------------+---+--------+-----------+--------+----+--------------------+----------+-----------+--------------------+
|               name| id|nametype|   recclass|mass (g)|fall|                year|    reclat|    reclong|         GeoLocation|
+-------------------+---+--------+-----------+--------+----+--------------------+----------+-----------+--------------------+
|             Aachen|  1|   Valid|         L5|      21|Fell|01/01/1880 12:00:...| 50.775000|   6.083330|(50.775000, 6.083...|
|             Aarhus|  2|   Valid|         H6|     720|Fell|01/01/1951 12:00:...| 56.183330|  10.233330|(56.183330, 10.23...|
|               Abee|  6|   Valid|        EH4|  107000|Fell|01/01/1952 12:00:...| 54.216670|-113.000000|(54.216670, -113....|
|           Acapulco| 10|   Valid|Acapulcoite|    1914|Fell|01/01/1976 12:00:...| 16.883330| -99.900000|(16.883330, -99.9...|
|            Achiras|370|   Valid|         L6|     780|Fell|01/01/1902 12:00:...|-33.166670| -64.950000|(-33.166670, -

In [11]:
df =df.cols().cast([("id","integer")])
df.dtypes

from optimus.profiler.profiler import Profiler
profiler = Profiler()

In [12]:
df.dtypes

[('name', 'string'),
 ('id', 'int'),
 ('nametype', 'string'),
 ('recclass', 'string'),
 ('mass (g)', 'string'),
 ('fall', 'string'),
 ('year', 'string'),
 ('reclat', 'string'),
 ('reclong', 'string'),
 ('GeoLocation', 'string')]

In [19]:
df =df.cols().cast([('reclat','integer')])
df.hist(['reclat','id'])

TypeError: _cast() missing 2 required positional arguments: 'col_name' and 'attr'

In [15]:
p =Profiler.columns(df, ['recclass','id'])

<class 'dict'>


In [16]:
j(p)

{
  "columns": {
    "recclass": {
      "uniques_count": 466,
      "p_uniques": 1.0193367748709423,
      "missing_count": 0,
      "p_missing": 0.0,
      "frequency": [
        {
          "value": "L6",
          "count": 8285,
          "percentage": 18.12275789657888
        },
        {
          "value": "H5",
          "count": 7142,
          "percentage": 15.622539154781695
        },
        {
          "value": "L5",
          "count": 4796,
          "percentage": 10.490856592877767
        },
        {
          "value": "H6",
          "count": 4528,
          "percentage": 9.904628576428385
        },
        {
          "value": "H4",
          "count": 4211,
          "percentage": 9.211217079359525
        },
        {
          "value": "LL5",
          "count": 2766,
          "percentage": 6.050398110070873
        },
        {
          "value": "LL6",
          "count": 2043,
          "percentage": 4.468894916440633
        },
        {
          "value": "L4

In [ ]:
Profiler.dataset_info(df)

In [ ]:
Profiler.count_data_types(df,'*')

In [31]:
df.cols().count_uniques('reclat')

{'reclat': 138}